In [ ]:
# 라인 트랙킹 + 객체 인식 + 장애물 회피를 합친 코드 
# 1. 30 미팅 후 수정
from pop import Pilot
import time

cam = Pilot.Camera(width=300, height=300)
ac = Pilot.AutoCar()
OF = Pilot.Object_Follow(cam)
LF = Pilot.Track_Follow(camera=cam)
CA = Pilot.Collision_Avoid(cam)

# 저장된 모델 3개를 불러오기
LF.load_model("line_tracer_0126_7.pth")  # 라인트래킹 모델 로드
OF.load_model() # 객체 인식 모델 불러오기 ("ssd_mobilenet_v2_coco.engine"를 자동으로 불러옴)
CA.load_model("collision_final.pth") # 장애물 회피 모델 로드

LF.show()
OF.show()
CA.show()

LF.run()
CA.run()

ac.setSpeed(20)

# 장애물 회피 & 라인트래킹 하면서 주행
def drive(val_LF, val_CA):
    ac.forward()
    steer = val_LF['x']  # 라인트랙킹을 위한 좌우 조향 값
    steer * 1.55
    
    # 좌우 조정
    if steer > 1:
        steer = 1
    elif steer < -1:
        steer = -1
    
    ac.steering = steer
    
    # 장애물 회피: 장애물 인식 시 후진
    if val_CA >= 0.5:    # 0.5 초과 시 후진
        ac.backward()
    

# 3개가 동시에 감지되면 정지하는 임시 코드
def object_detection_stop(chair, person, umbrella):
    # 객체 인식: 객체가 화면 대비 사이즈 20% 이상일 때 정지
    if chair:
        chair_detected(chair)
    elif person:
        person_detected(person)
    elif umbrella:
        umbrella_detected(umbrella)
    

# 의자 감지 시 정지
def chair_detected(chair):
    if chair and chair['size_rate'] >= 0.30:
        print("의자가 감지되었습니다. 정지합니다. size_rate = ", chair['size_rate'])
        ac.stop()

# 사람 감지 시 정지
def person_detected(person):
    if person and person['size_rate'] >= 0.40:
        print("사람이 감지되었습니다. 정지합니다.")
        ac.stop()

# 우산 감지 시 정지
def umbrella_detected(umbrella):
    if umbrella and umbrella['size_rate'] >= 0.20:
        print("우산이 감지되었습니다. 정지합니다.")
        ac.stop()


try:
    while True: # 의자, 사람, 우산 객체를 탐지하는 루프 
        v1_OF_chair = OF.detect(index='chair') 
        v2_OF_person = OF.detect(index='person')
        v3_OF_umbrella = OF.detect(index='umbrella')

        # 객체 인식 시 정지
        if v1_OF_chair:
            chair_detected(v1_OF_chair)
        elif v2_OF_person:
            person_detected(v2_OF_person)
        elif v3_OF_umbrella:
            umbrella_detected(v3_OF_umbrella)
        else:
            # 세개 중 하나도 해당되는 것이 없으면 계속해서 주행 진행.
            val_LF = LF.run()
            val_CA = CA.run()
            drive(val_LF, val_CA)

# 키보드로 진행 멈춤
except KeyboardInterrupt:
    ac.stop()